In [1]:
import os
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict, Tuple

In [ ]:
# Load configuration from config.json
with open('../models/config.json', 'r') as f:
    config = json.load(f)

postprocess = config['postprocessing']

# Output and input paths from configuration
OUTPUT_PATH = postprocess['output_path']
INPUT_PATH_S = postprocess['input_path_s']
INPUT_PATH_D = postprocess['input_path_d']

# New Input paths
INPUT_PATH_WS = postprocess['input_path_ws']
INPUT_PATH_HD = postprocess['input_path_hd']
INPUT_PATH_D_OBJ = postprocess['input_path_d_obj']
INPUT_PATH_S_OBJ = postprocess['input_path_s_obj']
INPUT_PATH_D_OBJ_ACT = postprocess['input_path_d_obj_act']


# Display the loaded paths
print("Loaded paths:")
print(f"OUTPUT_PATH: {OUTPUT_PATH}")
print(f"INPUT_PATH_S: {INPUT_PATH_S}")
print(f"INPUT_PATH_D: {INPUT_PATH_D}")
print(f"INPUT_PATH_WS: {INPUT_PATH_WS}")
print(f"INPUT_PATH_HD: {INPUT_PATH_HD}")

special_case = 'USE_EXT_COST_10'


Loaded paths:
OUTPUT_PATH: ../data/postprocessing/
INPUT_PATH_S: ../data/output/stochastic/timeseries/
INPUT_PATH_D: ../data/output/deterministic/timeseries/
INPUT_PATH_WS: ../data/output/deterministic/cases/scenarios/
INPUT_PATH_HD: ../data/input/demands/


## Filename Generations Functions

In [3]:
def get_ws_solution_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the Wait and See solution objective values CSV file.
    """
    filename = f"d_scenarios_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def get_probabilities_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the probabilities JSON file.
    """
    filename = f"reduced_heat_demand_scenarios_{start_date}_to_{end_date}_{period}.json"
    return filename

def get_weighted_deterministic_ts_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the deterministic weighted time series CSV file.
    """
    filename = f"d_weighted_{start_date}_to_{end_date}_{period}_ts.csv"
    return filename


def get_stochastic_objective_values_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the stochastic objective values CSV file.
    """
    filename = f"s_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def get_stochastic_objective_values_special_filename(start_date: str, end_date: str, period: str, special_case: str) -> str:
    """
    Generate the filename for the stochastic objective values CSV file.
    """
    filename = f"s_{start_date}_to_{end_date}_{period}_{special_case}_obj.csv"
    return filename

def get_actual_deterministic_objective_values_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the actual deterministic objective values CSV file.
    """
    filename = f"d_actual_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def get_deterministic_objective_values_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the actual deterministic objective values CSV file.
    """
    filename = f"d_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename


## Data Loading Functions

In [4]:
def load_ws_solution_data(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the Wait and See solution objective values for a specific date.
    """
    filename = get_ws_solution_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded Wait and See solution data from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

def load_probabilities(date_str: str, period: str, input_path: str) -> Dict[str, float]:
    """
    Load probabilities from the JSON file for a specific date and period.
    Returns a dictionary {ScenarioName: Probability}
    """
    filename = get_probabilities_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        print(f"Loaded probabilities from {filepath}")
        # Extract probabilities
        probabilities = {}
        for scenario_name, scenario_data in data.items():
            probabilities[scenario_name] = scenario_data.get('Probability', 0)
        return probabilities
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return {}
    
def load_weighted_deterministic_ts(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the deterministic weighted time series data for a specific date.
    """
    filename = get_weighted_deterministic_ts_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded deterministic weighted time series data from {filepath}")
        # Display the first few rows
        display(data.head())
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

def load_scenario_heat_demands(date_str: str, period: str, input_path: str) -> Dict[str, pd.Series]:
    """
    Load scenario heat demands from the JSON file for a specific date and period.
    Returns a dictionary {ScenarioName: pd.Series of heat demands}
    """
    filename = get_probabilities_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        print(f"Loaded scenario heat demands from {filepath}")
        # For each scenario, create a pandas Series with time steps and heat demands
        scenario_heat_demands = {}
        for scenario_name, scenario_data in data.items():
            # Remove 'Probability' key and adjust time steps to zero-based indexing
            heat_demand_data = {int(k) - 1: v for k, v in scenario_data.items() if k != 'Probability'}
            # Convert to Series and sort by time step
            heat_demand_series = pd.Series(heat_demand_data).sort_index()
            scenario_heat_demands[scenario_name] = heat_demand_series
        return scenario_heat_demands
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return {}
    

def get_weighted_deterministic_objective_value_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the filename for the deterministic weighted objective value CSV file.
    """
    filename = f"d_weighted_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def load_weighted_deterministic_objective_value(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the deterministic weighted objective value for a specific date.
    """
    filename = get_weighted_deterministic_objective_value_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded deterministic objective value from {filepath}")
        # Display the data
        display(data)
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    
def load_stochastic_objective_values(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the stochastic objective values for a specific date.
    """
    filename = get_stochastic_objective_values_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded stochastic objective values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()
    
def load_stochastic_objective_special_values(date_str: str, period: str, input_path: str, special_case: str) -> pd.DataFrame:
    """
    Load the stochastic objective values for a specific date.
    """
    filename = get_stochastic_objective_values_special_filename(date_str, date_str, period, special_case)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded stochastic objective special case values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()

def load_actual_deterministic_objective_values(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the actual deterministic objective values for a specific date.
    """
    filename = get_actual_deterministic_objective_values_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded actual deterministic objective values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()
    
def load_deterministic_objective_values(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load the actual deterministic objective values for a specific date.
    """
    filename = get_deterministic_objective_values_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        data = pd.read_csv(filepath)
        print(f"Loaded actual deterministic objective values from {filepath}")
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()




# Calculations

In [5]:
# List of dates to process
dates_to_process = [
    datetime(2023, 3, 16),
    datetime(2023, 3, 28),
    datetime(2023, 4, 3),
    datetime(2023, 4, 9),
    datetime(2023, 4, 10),
    datetime(2023, 5, 1),
    datetime(2023, 5, 15),
    datetime(2023, 6, 19),
    datetime(2023, 6, 22),
    datetime(2023, 6, 26),
    datetime(2023, 7, 9),
    datetime(2023, 7, 10),
    # Add mo
]

# Period for the analysis
period = 'day'


## Compute weighted Wait-and-See Solution

In [6]:
# Initialize a list to store the results
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")
    
    # Load Wait and See solution data
    ws_data = load_ws_solution_data(date_str, period, INPUT_PATH_WS)
    if ws_data.empty:
        print(f"No Wait and See solution data available for date {date_str}.")
        continue
    
    # Display the loaded Wait and See solution data
    print("Wait and See Objective Values:")
    display(ws_data)
    
    # Load probabilities
    probabilities = load_probabilities(date_str, period, INPUT_PATH_HD)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        continue
    
    # Display the probabilities
    print("Probabilities:")
    probabilities_df = pd.DataFrame.from_dict(probabilities, orient='index', columns=['Probability'])
    display(probabilities_df)
    
    # Merge ws_data with probabilities
    ws_data['Probability'] = ws_data['Scenario'].map(probabilities)
    
    # Check for missing probabilities
    if ws_data['Probability'].isnull().any():
        print("Some scenarios do not have associated probabilities.")
        display(ws_data[ws_data['Probability'].isnull()])
        continue
    
    # Compute weighted objective value
    ws_data['WeightedObjective'] = ws_data['ObjectiveValue'] * ws_data['Probability']
    
    # Print intermediate calculations
    print("Intermediate Calculations:")
    display(ws_data)
    
    # Sum the weighted objectives
    weighted_objective_value = ws_data['WeightedObjective'].sum()
    
    # Store the result
    results.append({
        'Date': date.strftime('%Y-%m-%d'),
        'Weighted Objective Value': weighted_objective_value
    })



Processing date: 20230316
Loaded Wait and See solution data from ../data/output/deterministic/cases/scenarios/d_scenarios_20230316_to_20230316_day_obj.csv
Wait and See Objective Values:


,Scenario,ObjectiveValue
0,Scenario1,-345.092638
1,Scenario2,-340.315156
2,Scenario3,-339.452650
3,Scenario4,-337.705625
4,Scenario5,-339.418949
5,Scenario6,-339.215562
6,Scenario7,-339.293524
7,Scenario8,-339.744750
8,Scenario9,-347.346345
9,Scenario10,-335.743155


Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Probabilities:


,Probability
Scenario1,0.093211
Scenario2,0.041917
Scenario3,0.073168
Scenario4,0.164225
Scenario5,0.096919
Scenario6,0.165247
Scenario7,0.048252
Scenario8,0.114138
Scenario9,0.088206
Scenario10,0.114716


Intermediate Calculations:


...

,Scenario,ObjectiveValue
0,Scenario1,-124.799953
1,Scenario2,-99.909010
2,Scenario3,-156.889541
3,Scenario4,-125.736980
4,Scenario5,-135.947097
5,Scenario6,-141.546969
6,Scenario7,-141.796380
7,Scenario8,-155.875290
8,Scenario9,-125.029847
9,Scenario10,-132.774851


Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Probabilities:


,Probability
Scenario1,0.157475
Scenario2,0.055415
Scenario3,0.134182
Scenario4,0.115740
Scenario5,0.069091
Scenario6,0.091832
Scenario7,0.110580
Scenario8,0.066341
Scenario9,0.130086
Scenario10,0.069256


Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-124.799953,0.157475,-19.652920
1,Scenario2,-99.909010,0.055415,-5.536489
2,Scenario3,-156.889541,0.134182,-21.051782
3,Scenario4,-125.736980,0.115740,-14.552808
4,Scenario5,-135.947097,0.069091,-9.392754
5,Scenario6,-141.546969,0.091832,-12.998544
6,Scenario7,-141.796380,0.110580,-15.679852
7,Scenario8,-155.875290,0.066341,-10.340972
8,Scenario9,-125.029847,0.130086,-16.264626
9,Scenario10,-132.774851,0.069256,-9.195517


#### Final Results Wait-and-See Solution

In [7]:
# Create a DataFrame of results
results_df = pd.DataFrame(results)
results_df['Weighted Objective Value'] = results_df['Weighted Objective Value'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df)



Final Results:


,Date,Weighted Objective Value
0,2023-03-16,-339.981554
1,2023-03-28,-348.433229
2,2023-04-03,-359.855693
3,2023-04-09,-375.489945
4,2023-04-10,-320.421686
5,2023-05-01,-287.570051
6,2023-05-15,-176.860330
7,2023-06-19,-127.705254
8,2023-06-22,-126.010359
9,2023-06-26,-124.417849


## Compute Heat Demand Difference for the calculation of EEV

In [8]:
def compute_costs_from_differences(date_str: str, period: str, input_path_det_ts: str, input_path_hd: str):
    """
    Compute the differences and costs between deterministic and scenario heat demands for a specific date.
    Returns:
        - total_costs_df: DataFrame containing total costs per scenario.
        - weighted_total_cost: The total weighted cost for the date.
    """
    # Load deterministic weighted time series data
    det_ts_data = load_weighted_deterministic_ts(date_str, period, input_path_det_ts)
    if det_ts_data.empty:
        print(f"No deterministic weighted time series data available for date {date_str}.")
        return None, None

    # Extract the deterministic heat demand
    det_heat_demand = det_ts_data['heat_demand']
    det_heat_demand.name = 'deterministic_heat_demand'

    # Load scenario heat demands
    scenario_heat_demands = load_scenario_heat_demands(date_str, period, input_path_hd)
    if not scenario_heat_demands:
        print(f"No scenario heat demands available for date {date_str}.")
        return None, None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None, None

    # Create a DataFrame to store costs
    costs_df = pd.DataFrame({
        't': det_ts_data['t'],
        'deterministic_heat_demand': det_heat_demand
    })

    total_costs = {}
    weighted_total_cost = 0.0

    # For each scenario, compute differences, costs, and total costs
    for scenario_name, scenario_hd_series in scenario_heat_demands.items():
        # Align scenario_hd_series with det_heat_demand
        scenario_hd_series = scenario_hd_series.reindex(det_heat_demand.index)
        scenario_hd_series.name = f"{scenario_name}_heat_demand"

        # Compute difference
        difference = det_heat_demand - scenario_hd_series
        difference.name = f"{scenario_name}_difference"

        # Compute cost per time step
        cost = difference.abs() * 0.01
        cost.name = f"{scenario_name}_cost"

        # Sum costs over time steps
        total_cost = cost.sum()

        # Get probability for the scenario
        probability = probabilities.get(scenario_name, 0)

        # Compute weighted cost
        weighted_cost = total_cost * probability

        # Add to total weighted cost
        weighted_total_cost += weighted_cost

        # Store total cost per scenario
        total_costs[scenario_name] = total_cost

        # Optionally, add to costs_df for detailed output
        costs_df[f"{scenario_name}_cost"] = cost.values

    # Create DataFrame for total costs per scenario
    total_costs_df = pd.DataFrame(list(total_costs.items()), columns=['Scenario', 'Total Cost'])

    # Display total costs per scenario
    print(f"Total Costs for date {date_str}:")
    display(total_costs_df)

    print(f"Weighted Total Cost for date {date_str}: {weighted_total_cost:.6f}")

    return total_costs_df, weighted_total_cost


In [9]:
def compute_costs_from_differences_special_case(date_str: str, period: str, input_path_det_ts: str, input_path_hd: str):
    """
    Compute the differences and costs between deterministic and scenario heat demands for a specific date,
    including additional costs when heat storage content exceeds a threshold.
    Returns:
        - total_costs_df: DataFrame containing total costs per scenario.
        - weighted_total_cost: The total weighted cost for the date.
    """
    # Load deterministic weighted time series data
    det_ts_data = load_weighted_deterministic_ts(date_str, period, input_path_det_ts)
    if det_ts_data.empty:
        print(f"No deterministic weighted time series data available for date {date_str}.")
        return None, None

    # Extract the deterministic heat demand
    det_heat_demand = det_ts_data['heat_demand']
    det_heat_demand.name = 'deterministic_heat_demand'

    # Extract the deterministic heat storage content
    if 'heat_storage1.heat_capacity' in det_ts_data.columns:
        det_heat_storage_content = det_ts_data['heat_storage1.heat_capacity']
    else:
        print(f"Heat storage content data not available in deterministic time series data for date {date_str}.")
        return None, None
    det_heat_storage_content.name = 'heat_storage1.heat_capacity'

    # Load scenario heat demands
    scenario_heat_demands = load_scenario_heat_demands(date_str, period, input_path_hd)
    if not scenario_heat_demands:
        print(f"No scenario heat demands available for date {date_str}.")
        return None, None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None, None

    # Create a DataFrame to store costs
    costs_df = pd.DataFrame({
        't': det_ts_data['t'],
        'deterministic_heat_demand': det_heat_demand,
        'deterministic_heat_storage_content': det_heat_storage_content,
    })

    total_costs = {}
    weighted_total_cost = 0.0

    # For each scenario, compute differences, costs, and total costs
    for scenario_name, scenario_hd_series in scenario_heat_demands.items():
        # Align scenario_hd_series with det_heat_demand
        scenario_hd_series = scenario_hd_series.reindex(det_heat_demand.index)
        scenario_hd_series.name = f"{scenario_name}_heat_demand"

        # Compute difference
        difference = det_heat_demand - scenario_hd_series
        difference.name = f"{scenario_name}_difference"

        # Compute adjusted storage content
        adjusted_storage_content = det_heat_storage_content + difference
        adjusted_storage_content.name = f"{scenario_name}_adjusted_storage_content"

        # Check if adjusted storage content exceeds 400
        exceeds_capacity = adjusted_storage_content > 400
        additional_cost = exceeds_capacity.astype(float) * 10  # 10 if exceeds, 0 otherwise

        # Compute cost per time step
        cost = difference.abs() * 0.01 + additional_cost
        cost.name = f"{scenario_name}_cost"

        # Sum costs over time steps
        total_cost = cost.sum()

        # Get probability for the scenario
        probability = probabilities.get(scenario_name, 0)

        # Compute weighted cost
        weighted_cost = total_cost * probability

        # Add to total weighted cost
        weighted_total_cost += weighted_cost

        # Store total cost per scenario
        total_costs[scenario_name] = total_cost

        # Optionally, add to costs_df for detailed output
        # costs_df[f"{scenario_name}_cost"] = cost.values

    # Create DataFrame for total costs per scenario
    total_costs_df = pd.DataFrame(list(total_costs.items()), columns=['Scenario', 'Total Cost'])

    # Display total costs per scenario
    print(f"Total Costs for date {date_str}:")
    display(total_costs_df)

    print(f"Weighted Total Cost for date {date_str}: {weighted_total_cost:.6f}")

    return total_costs_df, weighted_total_cost


In [10]:
# Input paths
input_path_det_ts = INPUT_PATH_D  # Path to deterministic weighted time series
input_path_hd = INPUT_PATH_HD     # Path to heat demand scenarios

# Dictionary to store total costs for each date
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute costs and weighted total cost
    total_costs_df, weighted_total_cost = compute_costs_from_differences_special_case(
        date_str=date_str,
        period=period,
        input_path_det_ts=input_path_det_ts,
        input_path_hd=input_path_hd
    )

    if weighted_total_cost is not None:
        # Load deterministic objective value
        det_obj_value = load_weighted_deterministic_objective_value(date_str, period, INPUT_PATH_D_OBJ)
        if det_obj_value is not None:
            # det_obj_value is a DataFrame with one value
            det_obj_value = det_obj_value['ObjectiveValue'].iloc[0]
            print(f"Deterministic Objective Value for date {date_str}: {det_obj_value}")

            # Compute total cost
            total_cost = det_obj_value + weighted_total_cost

            # Store the result
            results.append({
                'Date': date.strftime('%Y-%m-%d'),
                'Total Cost': total_cost
            })

            print(f"Total Cost for date {date_str}: {total_cost:.6f}")
        else:
            print(f"Could not load deterministic objective value for date {date_str}.")
    else:
        print(f"Could not compute weighted total cost for date {date_str}.")


Processing date: 20230316
Loaded deterministic weighted time series data from ../data/output/deterministic/timeseries/d_weighted_20230316_to_20230316_day_ts.csv


,t,GAS_PRICE,POWER_PRICE,HEAT_PRICE,heat_demand,heat_storage1.initial_soc,chp1.bin,chp1.power,chp1.gas,chp1.heat,...,heat_grid.heat_supply,heat_grid.heat_feedin,arc01_expanded.power,arc02_expanded.power,arc03_expanded.heat,arc04_expanded.heat,arc05_expanded.heat,arc06_expanded.gas,arc07_expanded.gas,arc08_expanded.heat
0,1,0.1543,0.251,0.105,122.338281,320.0,1.0,37.218589,117.039587,68.0,...,13.661719,136.000000,37.218589,38.890017,68.0,68.0,0.000000,0.000000,117.039587,0.000000
1,2,0.1543,0.251,0.105,134.504272,320.0,1.0,37.218589,117.039587,68.0,...,1.495728,136.000000,37.218589,38.890017,68.0,68.0,0.000000,0.000000,117.039587,0.000000
2,3,0.1543,0.251,0.105,197.239009,320.0,1.0,37.218589,117.039587,68.0,...,61.182393,258.421402,37.218589,38.890017,68.0,68.0,122.421402,113.583715,117.039587,0.000000
3,4,0.1543,0.251,0.105,331.596286,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,331.596286,37.218589,38.890017,68.0,68.0,174.000000,162.464986,117.039587,21.596286
4,5,0.1543,0.251,0.105,231.966560,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,231.966560,37.218589,38.890017,68.0,68.0,95.966560,88.512341,117.039587,0.000000


Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Total Costs for date 20230316:


,Scenario,Total Cost
0,Scenario1,13.220186
1,Scenario2,23.225086
2,Scenario3,14.317285
3,Scenario4,2.564928
4,Scenario5,23.652522
5,Scenario6,13.123079
6,Scenario7,33.916078
7,Scenario8,23.328031
8,Scenario9,43.130369
9,Scenario10,13.441913


Weighted Total Cost for date 20230316: 17.781033
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230316_to_20230316_day_obj.csv


...

Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Total Costs for date 20230710:


,Scenario,Total Cost
0,Scenario1,2.852600
1,Scenario2,3.922728
2,Scenario3,3.420707
3,Scenario4,3.605837
4,Scenario5,3.680253
5,Scenario6,3.037037
6,Scenario7,2.914994
7,Scenario8,2.621812
8,Scenario9,13.167353
9,Scenario10,13.460530


Weighted Total Cost for date 20230710: 5.217493
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230710_to_20230710_day_obj.csv


,ObjectiveValue
0,-134.913504


Deterministic Objective Value for date 20230710: -134.91350375977672
Total Cost for date 20230710: -129.696011


In [11]:

# Create a DataFrame of results
results_df_eev_special = pd.DataFrame(results)
results_df_eev_special['Total Cost'] = results_df_eev_special['Total Cost'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_eev_special)

# Save the EEV results to a CSV file
filename = "eev_special.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_eev_special.to_csv(output_filepath, index=False)




Final Results:


,Date,Total Cost
0,2023-03-16,-322.002572
1,2023-03-28,-345.961889
2,2023-04-03,-356.189548
3,2023-04-09,-372.243213
4,2023-04-10,-313.272666
5,2023-05-01,-274.582693
6,2023-05-15,-169.192157
7,2023-06-19,-118.521814
8,2023-06-22,-105.361630
9,2023-06-26,-97.688779


In [12]:
# Input paths
input_path_det_ts = INPUT_PATH_D  # Path to deterministic weighted time series
input_path_hd = INPUT_PATH_HD     # Path to heat demand scenarios

# Dictionary to store total costs for each date
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute costs and weighted total cost
    total_costs_df, weighted_total_cost = compute_costs_from_differences(
        date_str=date_str,
        period=period,
        input_path_det_ts=input_path_det_ts,
        input_path_hd=input_path_hd
    )

    if weighted_total_cost is not None:
        # Load deterministic objective value
        det_obj_value = load_weighted_deterministic_objective_value(date_str, period, INPUT_PATH_D_OBJ)
        if det_obj_value is not None:
            # det_obj_value is a DataFrame with one value
            det_obj_value = det_obj_value['ObjectiveValue'].iloc[0]
            print(f"Deterministic Objective Value for date {date_str}: {det_obj_value}")

            # Compute total cost
            total_cost = det_obj_value + weighted_total_cost

            # Store the result
            results.append({
                'Date': date.strftime('%Y-%m-%d'),
                'Total Cost': total_cost
            })

            print(f"Total Cost for date {date_str}: {total_cost:.6f}")
        else:
            print(f"Could not load deterministic objective value for date {date_str}.")
    else:
        print(f"Could not compute weighted total cost for date {date_str}.")


Processing date: 20230316
Loaded deterministic weighted time series data from ../data/output/deterministic/timeseries/d_weighted_20230316_to_20230316_day_ts.csv


,t,GAS_PRICE,POWER_PRICE,HEAT_PRICE,heat_demand,heat_storage1.initial_soc,chp1.bin,chp1.power,chp1.gas,chp1.heat,...,heat_grid.heat_supply,heat_grid.heat_feedin,arc01_expanded.power,arc02_expanded.power,arc03_expanded.heat,arc04_expanded.heat,arc05_expanded.heat,arc06_expanded.gas,arc07_expanded.gas,arc08_expanded.heat
0,1,0.1543,0.251,0.105,122.338281,320.0,1.0,37.218589,117.039587,68.0,...,13.661719,136.000000,37.218589,38.890017,68.0,68.0,0.000000,0.000000,117.039587,0.000000
1,2,0.1543,0.251,0.105,134.504272,320.0,1.0,37.218589,117.039587,68.0,...,1.495728,136.000000,37.218589,38.890017,68.0,68.0,0.000000,0.000000,117.039587,0.000000
2,3,0.1543,0.251,0.105,197.239009,320.0,1.0,37.218589,117.039587,68.0,...,61.182393,258.421402,37.218589,38.890017,68.0,68.0,122.421402,113.583715,117.039587,0.000000
3,4,0.1543,0.251,0.105,331.596286,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,331.596286,37.218589,38.890017,68.0,68.0,174.000000,162.464986,117.039587,21.596286
4,5,0.1543,0.251,0.105,231.966560,320.0,1.0,37.218589,117.039587,68.0,...,0.000000,231.966560,37.218589,38.890017,68.0,68.0,95.966560,88.512341,117.039587,0.000000


Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Total Costs for date 20230316:


,Scenario,Total Cost
0,Scenario1,3.220186
1,Scenario2,3.225086
2,Scenario3,4.317285
3,Scenario4,2.564928
4,Scenario5,3.652522
5,Scenario6,3.123079
6,Scenario7,3.916078
7,Scenario8,3.328031
8,Scenario9,3.130369
9,Scenario10,3.441913


Weighted Total Cost for date 20230316: 3.282309
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230316_to_20230316_day_obj.csv


...

Loaded scenario heat demands from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Total Costs for date 20230710:


,Scenario,Total Cost
0,Scenario1,2.852600
1,Scenario2,3.922728
2,Scenario3,3.420707
3,Scenario4,3.605837
4,Scenario5,3.680253
5,Scenario6,3.037037
6,Scenario7,2.914994
7,Scenario8,2.621812
8,Scenario9,3.167353
9,Scenario10,3.460530


Weighted Total Cost for date 20230710: 3.224069
Loaded deterministic objective value from ../data/output/deterministic/objectives/d_weighted_20230710_to_20230710_day_obj.csv


,ObjectiveValue
0,-134.913504


Deterministic Objective Value for date 20230710: -134.91350375977672
Total Cost for date 20230710: -131.689435


## EEV 

In [13]:
# Create a DataFrame of results
results_df_eev = pd.DataFrame(results)
results_df_eev['Total Cost'] = results_df_eev['Total Cost'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_eev)




Final Results:


,Date,Total Cost
0,2023-03-16,-336.501295
1,2023-03-28,-345.961889
2,2023-04-03,-356.189548
3,2023-04-09,-372.243213
4,2023-04-10,-318.034454
5,2023-05-01,-284.986767
6,2023-05-15,-169.192157
7,2023-06-19,-126.208851
8,2023-06-22,-115.353344
9,2023-06-26,-116.644104


In [14]:
def compute_weighted_stochastic_objective_value(date_str: str, period: str, input_path_s: str, input_path_hd: str) -> float:
    """
    Compute the weighted stochastic objective value for a specific date.
    """
    # Load stochastic objective values
    sto_obj_values_df = load_stochastic_objective_values(date_str, period, input_path_s)
    if sto_obj_values_df.empty:
        print(f"No stochastic objective values data available for date {date_str}.")
        return None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None

    # Clean up column names if necessary
    # Sometimes the first column might have a stray ':' character
    sto_obj_values_df.columns = [col.strip(':') for col in sto_obj_values_df.columns]

    # Merge sto_obj_values_df with probabilities
    sto_obj_values_df['Probability'] = sto_obj_values_df['Scenario'].map(probabilities)

    # Check for missing probabilities
    if sto_obj_values_df['Probability'].isnull().any():
        print("Some scenarios do not have associated probabilities.")
        display(sto_obj_values_df[sto_obj_values_df['Probability'].isnull()])
        return None

    # Compute weighted objective value
    sto_obj_values_df['WeightedObjective'] = sto_obj_values_df['ObjectiveValue'] * sto_obj_values_df['Probability']

    # Display intermediate calculations
    print("Intermediate Calculations:")
    display(sto_obj_values_df)

    # Sum the weighted objectives
    weighted_objective_value = sto_obj_values_df['WeightedObjective'].sum()

    return weighted_objective_value


In [15]:
def compute_weighted_stochastic_objective_special_value(date_str: str, period: str, input_path_s: str, input_path_hd: str, special_case: str) -> float:
    """
    Compute the weighted stochastic objective value for a specific date.
    """
    # Load stochastic objective values
    sto_obj_values_df = load_stochastic_objective_special_values(date_str, period, input_path_s, special_case)
    if sto_obj_values_df.empty:
        print(f"No stochastic objective values data available for date {date_str}.")
        return None

    # Load probabilities
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None

    # Clean up column names if necessary
    # Sometimes the first column might have a stray ':' character
    sto_obj_values_df.columns = [col.strip(':') for col in sto_obj_values_df.columns]

    # Merge sto_obj_values_df with probabilities
    sto_obj_values_df['Probability'] = sto_obj_values_df['Scenario'].map(probabilities)

    # Check for missing probabilities
    if sto_obj_values_df['Probability'].isnull().any():
        print("Some scenarios do not have associated probabilities.")
        display(sto_obj_values_df[sto_obj_values_df['Probability'].isnull()])
        return None

    # Compute weighted objective value
    sto_obj_values_df['WeightedObjective'] = sto_obj_values_df['ObjectiveValue'] * sto_obj_values_df['Probability']

    # Display intermediate calculations
    print("Intermediate Calculations:")
    display(sto_obj_values_df)

    # Sum the weighted objectives
    weighted_objective_value = sto_obj_values_df['WeightedObjective'].sum()

    return weighted_objective_value


In [16]:
# Input paths
input_path_s_obj = INPUT_PATH_S_OBJ  # Path to stochastic objective values
input_path_hd = INPUT_PATH_HD  # Path to heat demand scenarios (probabilities)

# Initialize a list to store the results
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute weighted stochastic objective value
    weighted_sto_obj_value = compute_weighted_stochastic_objective_value(
        date_str=date_str,
        period=period,
        input_path_s=input_path_s_obj,
        input_path_hd=input_path_hd
    )

    if weighted_sto_obj_value is not None:
        # Store the result
        results.append({
            'Date': date.strftime('%Y-%m-%d'),
            'Weighted Stochastic Objective Value': weighted_sto_obj_value
        })
        print(f"Weighted Stochastic Objective Value for date {date_str}: {weighted_sto_obj_value:.6f}")
    else:
        print(f"Could not compute weighted stochastic objective value for date {date_str}.")


Processing date: 20230316
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230316_to_20230316_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-326.915809,0.093211,-30.471995
1,Scenario2,-327.040740,0.041917,-13.708444
2,Scenario3,-325.951884,0.073168,-23.849111
3,Scenario4,-327.274089,0.164225,-53.746681
4,Scenario5,-326.848278,0.096919,-31.677880
5,Scenario6,-326.980714,0.165247,-54.032743
6,Scenario7,-326.691768,0.048252,-15.763669
7,Scenario8,-326.885557,0.114138,-37.310148
8,Scenario9,-326.915483,0.088206,-28.835980
9,Scenario10,-326.848067,0.114716,-37.494821


Weighted Stochastic Objective Value for date 20230316: -326.891473

Processing date: 20230328
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230328_to_20230328_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230328_to_20230328_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-339.124442,0.057758,-19.587297
1,Scenario2,-340.432339,0.126084,-42.923029
2,Scenario3,-340.055679,0.074060,-25.184358
3,Scenario4,-339.474604,0.137439,-46.656889
4,Scenario5,-339.882296,0.181896,-61.823087
5,Scenario6,-340.267599,0.073522,-25.017144
6,Scenario7,-340.798173,0.100274,-34.173173
7,Scenario8,-339.836103,0.057625,-19.583008
8,Scenario9,-339.352259,0.107217,-36.384223
9,Scenario10,-340.063089,0.084127,-28.608362


Weighted Stochastic Objective Value for date 20230328: -339.940569

Processing date: 20230403
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230403_to_20230403_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230403_to_20230403_day.json
Intermediate Calculations:


...

Weighted Stochastic Objective Value for date 20230626: -145.418309

Processing date: 20230709
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230709_to_20230709_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230709_to_20230709_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-153.263132,0.063022,-9.658976
1,Scenario2,-152.232274,0.083171,-12.661328
2,Scenario3,-151.903239,0.133864,-20.334321
3,Scenario4,-151.941713,0.077075,-11.710919
4,Scenario5,-152.881513,0.077908,-11.910673
5,Scenario6,-151.522232,0.121064,-18.343904
6,Scenario7,-151.365279,0.078744,-11.919158
7,Scenario8,-152.986755,0.165071,-25.253662
8,Scenario9,-151.975862,0.145044,-22.043211
9,Scenario10,-152.860716,0.055037,-8.412937


Weighted Stochastic Objective Value for date 20230709: -152.249089

Processing date: 20230710
Loaded stochastic objective values from ../data/output/stochastic/objectives/s_20230710_to_20230710_day_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-152.079868,0.157475,-23.948835
1,Scenario2,-150.898947,0.055415,-8.362112
2,Scenario3,-151.771827,0.134182,-20.365075
3,Scenario4,-150.457646,0.115740,-17.413980
4,Scenario5,-150.811872,0.069091,-10.419779
5,Scenario6,-151.196893,0.091832,-13.884716
6,Scenario7,-151.380801,0.110580,-16.739698
7,Scenario8,-151.829379,0.066341,-10.072561
8,Scenario9,-151.392273,0.130086,-19.694007
9,Scenario10,-150.347923,0.069256,-10.412566


Weighted Stochastic Objective Value for date 20230710: -151.313329


In [17]:
# Create a DataFrame of results
results_df_rp = pd.DataFrame(results)
results_df_rp['Weighted Stochastic Objective Value'] = results_df_rp['Weighted Stochastic Objective Value'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_rp)



Final Results:


,Date,Weighted Stochastic Objective Value
0,2023-03-16,-326.891473
1,2023-03-28,-339.940569
2,2023-04-03,-346.039721
3,2023-04-09,-364.383643
4,2023-04-10,-342.530242
5,2023-05-01,-298.840128
6,2023-05-15,-195.907414
7,2023-06-19,-150.379228
8,2023-06-22,-153.941702
9,2023-06-26,-145.418309


## Special Case RP

In [18]:
# Input paths
input_path_s_obj = INPUT_PATH_S_OBJ  # Path to stochastic objective values
input_path_hd = INPUT_PATH_HD  # Path to heat demand scenarios (probabilities)

# Initialize a list to store the results
results = []

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    # Compute weighted stochastic objective value
    weighted_sto_obj_special_value = compute_weighted_stochastic_objective_special_value(
        date_str=date_str,
        period=period,
        input_path_s=input_path_s_obj,
        input_path_hd=input_path_hd,
        special_case=special_case
    )

    if weighted_sto_obj_special_value is not None:
        # Store the result
        results.append({
            'Date': date.strftime('%Y-%m-%d'),
            'Weighted Stochastic Objective Value': weighted_sto_obj_special_value
        })
        print(f"Weighted Stochastic Objective Value for date {date_str} and case {special_case}: {weighted_sto_obj_special_value:.6f}")
    else:
        print(f"Could not compute weighted stochastic objective value for date {date_str}.")


Processing date: 20230316
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230316_to_20230316_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230316_to_20230316_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-326.915809,0.093211,-30.471995
1,Scenario2,-327.040740,0.041917,-13.708444
2,Scenario3,-325.951884,0.073168,-23.849111
3,Scenario4,-327.274089,0.164225,-53.746681
4,Scenario5,-326.848278,0.096919,-31.677880
5,Scenario6,-326.980714,0.165247,-54.032743
6,Scenario7,-326.691768,0.048252,-15.763669
7,Scenario8,-326.885557,0.114138,-37.310148
8,Scenario9,-326.915483,0.088206,-28.835980
9,Scenario10,-326.848067,0.114716,-37.494821


Weighted Stochastic Objective Value for date 20230316 and case USE_EXT_COST_10: -326.891473

Processing date: 20230328
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230328_to_20230328_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230328_to_20230328_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-339.124442,0.057758,-19.587297
1,Scenario2,-340.432339,0.126084,-42.923029
2,Scenario3,-340.055679,0.074060,-25.184358
3,Scenario4,-339.474604,0.137439,-46.656889
4,Scenario5,-339.882296,0.181896,-61.823087
5,Scenario6,-340.267599,0.073522,-25.017144
6,Scenario7,-340.798173,0.100274,-34.173173
7,Scenario8,-339.836103,0.057625,-19.583008
8,Scenario9,-339.352259,0.107217,-36.384223
9,Scenario10,-340.063089,0.084127,-28.608362


Weighted Stochastic Objective Value for date 20230328 and case USE_EXT_COST_10: -339.940569

Processing date: 20230403
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230403_to_20230403_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230403_to_20230403_day.json
Intermediate Calculations:


...

Weighted Stochastic Objective Value for date 20230626 and case USE_EXT_COST_10: -144.829928

Processing date: 20230709
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230709_to_20230709_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230709_to_20230709_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-152.287756,0.063022,-9.597506
1,Scenario2,-151.256897,0.083171,-12.580204
2,Scenario3,-150.927863,0.133864,-20.203754
3,Scenario4,-150.966336,0.077075,-11.635742
4,Scenario5,-151.906136,0.077908,-11.834683
5,Scenario6,-150.546856,0.121064,-18.225821
6,Scenario7,-150.389902,0.078744,-11.842353
7,Scenario8,-152.011378,0.165071,-25.092656
8,Scenario9,-151.000485,0.145044,-21.901738
9,Scenario10,-151.885339,0.055037,-8.359255


Weighted Stochastic Objective Value for date 20230709 and case USE_EXT_COST_10: -151.273712

Processing date: 20230710
Loaded stochastic objective special case values from ../data/output/stochastic/objectives/s_20230710_to_20230710_day_USE_EXT_COST_10_obj.csv
Loaded probabilities from ../data/input/demands/reduced_heat_demand_scenarios_20230710_to_20230710_day.json
Intermediate Calculations:


,Scenario,ObjectiveValue,Probability,WeightedObjective
0,Scenario1,-151.863836,0.157475,-23.914815
1,Scenario2,-150.682914,0.055415,-8.350140
2,Scenario3,-151.555794,0.134182,-20.336088
3,Scenario4,-150.241613,0.115740,-17.388976
4,Scenario5,-150.595839,0.069091,-10.404853
5,Scenario6,-140.980860,0.091832,-12.946557
6,Scenario7,-151.164768,0.110580,-16.715809
7,Scenario8,-151.613346,0.066341,-10.058229
8,Scenario9,-151.176240,0.130086,-19.665904
9,Scenario10,-150.131890,0.069256,-10.397604


Weighted Stochastic Objective Value for date 20230710 and case USE_EXT_COST_10: -150.178976


In [19]:
# Create a DataFrame of results
results_df_rp_special = pd.DataFrame(results)
results_df_rp_special['Weighted Stochastic Objective Value'] = results_df_rp_special['Weighted Stochastic Objective Value'].round(6)  # Round for readability

# Display the final results
print("\nFinal Results:")
display(results_df_rp_special)



Final Results:


,Date,Weighted Stochastic Objective Value
0,2023-03-16,-326.891473
1,2023-03-28,-339.940569
2,2023-04-03,-345.915502
3,2023-04-09,-364.085419
4,2023-04-10,-344.464910
5,2023-05-01,-293.404160
6,2023-05-15,-195.843578
7,2023-06-19,-150.354367
8,2023-06-22,-153.905792
9,2023-06-26,-144.829928


In [20]:
# Save the EEV results to a CSV file
filename = "eev.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_eev.to_csv(output_filepath, index=False)


# Save the WS results to a CSV file
filename = "ws.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df.to_csv(output_filepath, index=False)


# Save the RP results to a CSV file
filename = "rp.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_rp.to_csv(output_filepath, index=False)


In [21]:
# Merge the dataframes on 'Date'
merged_df = pd.merge(results_df, results_df_rp, on='Date')
# Calculate EVPI
merged_df['EVPI'] = merged_df['Weighted Stochastic Objective Value'] - merged_df['Weighted Objective Value']

# Rename columns
merged_df.rename(columns={
    'Weighted Objective Value': 'WS',
    'Weighted Stochastic Objective Value': 'RP'
}, inplace=True)
# Create the results DataFrame with 'Date' and 'EVPI'
results_df_evpi = merged_df[['Date', 'EVPI']]

# Round the values in merged_df to two decimal places
merged_df = merged_df.round(2)

# Display the results
print(results_df_evpi)


          Date       EVPI
0   2023-03-16  13.090081
1   2023-03-28   8.492660
2   2023-04-03  13.815972
3   2023-04-09  11.106302
4   2023-04-10 -22.108556
5   2023-05-01 -11.270077
6   2023-05-15 -19.047084
7   2023-06-19 -22.673974
8   2023-06-22 -27.931343
9   2023-06-26 -21.000460
10  2023-07-09 -15.966399
11  2023-07-10 -16.647066


In [22]:
# Save the EVPI results to a CSV file
filename = "evpi.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

results_df_evpi.to_csv(output_filepath, index=False)

# Save the all results to a CSV file
filename = "all.csv"  # Specify your desired filename
output_filepath = os.path.join(OUTPUT_PATH, filename)

merged_df.to_csv(output_filepath, index=False)

In [23]:
def collect_actual_objective_values(dates_to_process: List[datetime], period: str, input_path: str) -> pd.DataFrame:
    """
    Collect the actual deterministic objective values for a list of dates.

    Parameters:
    - dates_to_process: List of datetime objects representing the dates to process.
    - period: The period string used in the filenames (e.g., 'day', 'week').
    - input_path: The path where the objective value CSV files are located.

    Returns:
    - A pandas DataFrame with columns 'Date' and 'Actual Value'.
    """
    import os
    import pandas as pd
    from datetime import datetime

    actual_values = []

    for date in dates_to_process:
        date_str = date.strftime('%Y%m%d')
        # Use your existing function to load the data
        data = load_actual_deterministic_objective_values(date_str, period, input_path)
        if not data.empty:
            # Assuming the CSV has columns 'Scenario' and 'ObjectiveValue'
            actual_row = data[data['Scenario'] == 'actual']
            if not actual_row.empty:
                objective_value = actual_row['ObjectiveValue'].iloc[0]
                actual_values.append({
                    'Date': date.strftime('%Y-%m-%d'),
                    'Actual Value': objective_value
                })
            else:
                print(f"No 'actual' scenario found in data for date {date_str}")
        else:
            print(f"Data not available for date {date_str}")

    # Create a DataFrame from the results
    actual_values_df = pd.DataFrame(actual_values)
    # Sort by date
    actual_values_df = actual_values_df.sort_values('Date').reset_index(drop=True)
    return actual_values_df


In [24]:
input_path = INPUT_PATH_D_OBJ_ACT  # Path to actual deterministic objective values

actual_values_df = collect_actual_objective_values(dates_to_process, period, input_path)

# Display the DataFrame
print("\nActual Objective Values:")
print(actual_values_df)

# Optionally, write the DataFrame to a CSV file
filename = "actual_objective_values.csv"
output_filepath = os.path.join(OUTPUT_PATH, filename)
actual_values_df.to_csv(output_filepath, index=False)
print(f"\nActual objective values have been written to {output_filepath}")

Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230316_to_20230316_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230328_to_20230328_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230403_to_20230403_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230409_to_20230409_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230410_to_20230410_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230501_to_20230501_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/cases/actual/d_actual_20230515_to_20230515_day_obj.csv
Loaded actual deterministic objective values from ../data/outp

In [25]:
def collect_objective_values(dates_to_process: List[datetime], period: str, input_path: str) -> pd.DataFrame:
    """
    Collect the actual deterministic objective values for a list of dates.

    Parameters:
    - dates_to_process: List of datetime objects representing the dates to process.
    - period: The period string used in the filenames (e.g., 'day', 'week').
    - input_path: The path where the objective value CSV files are located.

    Returns:
    - A pandas DataFrame with columns 'Date' and 'Actual Value'.
    """
    import os
    import pandas as pd
    from datetime import datetime

    values = []

    for date in dates_to_process:
        date_str = date.strftime('%Y%m%d')
        # Use your existing function to load the data
        data = load_deterministic_objective_values(date_str, period, input_path)
        if not data.empty:
            if 'ObjectiveValue' in data.columns:
                objective_value = data['ObjectiveValue'].iloc[0]
                values.append({
                    'Date': date.strftime('%Y-%m-%d'),
                    'Objective Value': objective_value
                })
            else:
                print(f"Die Spalte 'ObjectiveValue' wurde in der Datei {filename} nicht gefunden.")
        else:
            print(f"Datei {filename} existiert nicht im Pfad {input_path}.")
            
            
           
    # Create a DataFrame from the results
    values_df = pd.DataFrame(values)
    # Sort by date
    values_df = values_df.sort_values('Date').reset_index(drop=True)
    return values_df


In [26]:
input_path = INPUT_PATH_D_OBJ  # Ersetzen Sie dies durch den tatsächlichen Pfad

# Aufrufen der Funktion
objective_values_df = collect_objective_values(dates_to_process, period, input_path)

# Anzeigen der Ergebnisse
print(objective_values_df)

# Optionally, write the DataFrame to a CSV file
filename = "deterministic_objective_values.csv"
output_filepath = os.path.join(OUTPUT_PATH, filename)
objective_values_df.to_csv(output_filepath, index=False)
print(f"\nObjective values have been written to {output_filepath}")

Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230316_to_20230316_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230328_to_20230328_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230403_to_20230403_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230409_to_20230409_day_obj.csv


Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230410_to_20230410_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230501_to_20230501_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230515_to_20230515_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230619_to_20230619_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230622_to_20230622_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230626_to_20230626_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230709_to_20230709_day_obj.csv
Loaded actual deterministic objective values from ../data/output/deterministic/objectives/d_20230710_to_20230710_day_obj.csv


In [27]:
# Merge the first two DataFrames on 'Date'
merged_df = pd.merge(objective_values_df, results_df_rp, on='Date', how='outer')
# Merge the result with the third DataFrame
merged_df = pd.merge(merged_df, actual_values_df, on='Date', how='outer')

# Rename columns
merged_df.rename(columns={
    'Objective Value': 'DM Objective Value',
    'Weighted Stochastic Objective Value': 'SM Objective Value',
    'Actual Value': 'Actual Objective Value'
}, inplace=True)

merged_df = merged_df.round(2)

# Display the merged DataFrame
print("Merged DataFrame:")
print(merged_df)

# Optionally, write the DataFrame to a CSV file
filename = "all_objective_values.csv"
output_filepath = os.path.join(OUTPUT_PATH, filename)
merged_df.to_csv(output_filepath, index=False)
print(f"\nObjective values have been written to {output_filepath}")

Merged DataFrame:
          Date  DM Objective Value  SM Objective Value  Actual Objective Value
0   2023-03-16             -331.45             -326.89                 -361.99
1   2023-03-28             -344.34             -339.94                 -374.83
2   2023-04-03             -350.83             -346.04                 -363.36
3   2023-04-09             -369.14             -364.38                 -325.20
4   2023-04-10             -351.35             -342.53                 -284.11
5   2023-05-01             -319.20             -298.84                 -278.94
6   2023-05-15             -204.75             -195.91                 -210.56
7   2023-06-19             -156.40             -150.38                 -142.04
8   2023-06-22             -159.80             -153.94                 -141.28
9   2023-06-26             -151.95             -145.42                 -151.59
10  2023-07-09             -159.84             -152.25                 -144.74
11  2023-07-10             -157.83